In [48]:
import numpy as np
import pandas as pd
import math
import random


import optuna
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import restore_demand_beta as dem

import warnings
warnings.filterwarnings('ignore')

In [49]:
sales = pd.read_csv('data/DFR_SALES.csv')[['PRODUCT_ID', 'location_id', 'PERIOD_DT', 'SALES_QTY', 'COST']]
sales.rename(columns = {'PRODUCT_ID':'product_id', 'location_id':'location_id', 'PERIOD_DT' : 'date', 'SALES_QTY': 'sales_qty', 'COST' : 'cost'}, inplace = True)


sales['date'] = pd.to_datetime(sales['date'])

In [50]:
sales

,product_id,location_id,date,sales_qty,cost
0,8000054507,5637144625,2018-04-01,13,717.50
1,8000054507,5637144625,2018-04-02,1,717.50
2,8000054507,5637144625,2018-04-12,4,717.50
3,8000054507,5637144625,2018-04-14,8,717.50
4,8000054507,5637144625,2018-04-15,1,717.50
...,...,...,...,...,...
24447,9000400501,9000136500,2020-03-10,4,4.43
24448,9000400501,9000136500,2020-03-11,1,4.43
24449,9000400501,9000136500,2020-03-12,1,4.43
24450,9000400501,9000136500,2020-03-15,1,4.43


In [51]:
stock = pd.read_csv('data/DFR_STOCK.csv')[['PRODUCT_ID', 'LOCATION_ID', 'PERIOD_DT', 'STOCK_QTY']]
stock.rename(columns = {'PRODUCT_ID':'product_id', 'LOCATION_ID':'location_id', 'PERIOD_DT' : 'date', 'STOCK_QTY': 'stock_qty'}, inplace = True)
stock['date'] = pd.to_datetime(stock['date'])

In [52]:
result = pd.merge(stock, sales, how="left", on=["product_id", "location_id", "date"])

In [53]:
result

,product_id,location_id,date,stock_qty,sales_qty,cost
0,8000054507,5637146288,2020-06-09,0,1.0,717.5000
1,8000054507,9000135750,2020-06-13,1,1.0,717.5000
2,8000054507,5637146288,2020-06-08,1,1.0,717.5000
3,8000145055,5637146303,2020-06-16,4,0.0,83.7728
4,9000883500,5637146282,2020-06-15,4,NaN,NaN
...,...,...,...,...,...,...
49867,8000089444,5637149414,2017-04-20,19,NaN,NaN
49868,8000089444,5637149414,2017-04-12,20,NaN,NaN
49869,8000089444,5637149414,2017-04-06,21,NaN,NaN
49870,9000271526,5637145298,2017-09-29,28,NaN,NaN


In [56]:
result.to_csv('resultData.csv', index=False)

In [65]:
resultData = pd.read_csv('resultData.csv')


In [66]:
resultData

,product_id,location_id,date,sales_qty,stock_qty,cost,promo
0,8000054507,5637146288,2020-06-09 00:00:00,1.0,0,717.5000,0
1,8000054507,9000135750,2020-06-13 00:00:00,1.0,1,717.5000,0
2,8000054507,5637146288,2020-06-08 00:00:00,1.0,1,717.5000,0
3,8000145055,5637146303,2020-06-16 00:00:00,0.0,4,83.7728,0
4,9000883500,5637146282,2020-06-15 00:00:00,NaN,4,NaN,0
...,...,...,...,...,...,...,...
50245,8000089444,5637149414,2017-04-20 00:00:00,NaN,19,NaN,1
50246,8000089444,5637149414,2017-04-12 00:00:00,NaN,20,NaN,1
50247,8000089444,5637149414,2017-04-06 00:00:00,NaN,21,NaN,1
50248,9000271526,5637145298,2017-09-29 00:00:00,NaN,28,NaN,1


In [59]:
promo = pd.read_csv('data/DFR_PROMO.csv')[['PRODUCT_ID', 'LOCATION_ID', 'PERIOD_START_DT', 'PERIOD_END_DT']]
promo.rename(columns = {'PRODUCT_ID':'product_id', 'LOCATION_ID':'location_id', 'PERIOD_START_DT' : 'start_date', 'PERIOD_END_DT': 'end_date'}, inplace = True)
promo['start_date'] = pd.to_datetime(promo['start_date'])
promo['end_date'] = pd.to_datetime(promo['end_date'])
promo

,product_id,location_id,start_date,end_date
0,8000054507,5637144625,2015-12-21,2015-12-28
1,8000054507,5637144625,2016-04-04,2016-05-02
2,8000054507,5637144625,2016-07-11,2016-07-18
3,8000054507,5637144625,2016-08-29,2016-10-03
4,8000054507,5637144625,2016-10-31,2017-01-02
...,...,...,...,...
3642,9000271526,9000135750,2019-12-16,2020-01-06
3643,9000271526,9000135750,2020-03-02,2020-03-09
3644,9000271526,9000136500,2019-11-11,2019-12-02
3645,9000271526,9000136500,2019-12-16,2020-01-06


In [60]:
import sqlite3

In [61]:
conn = sqlite3.connect(':memory:')

In [62]:
result.to_sql('result', conn, index=False)
promo.to_sql('promo', conn, index=False)


3647

In [63]:
qry = '''
    select  
        r.product_id,
        r.location_id,
        r.date,
        r.sales_qty,
        r.stock_qty,
        r.cost,
        case
            when p.start_date is not null and p.end_date is not null then 1
            else 0
        end as promo
    from result as r
    left join promo as p
        on r.date between p.start_date and p.end_date 
        and r.product_id = p.product_id
        and r.location_id = p.location_id
    '''
df = pd.read_sql_query(qry, conn)

In [ ]:
df = df.drop_duplicates()
df['date'] = pd.to_datetime(df['date'])
df

In [64]:
df.to_csv('resultData.csv', index=False)